In [6]:
from pathlib import Path
from dotenv import load_dotenv
import os

# LangChain 모듈
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

# 유사도 계산
from sentence_transformers import CrossEncoder
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# ─── 환경변수 로드 ─────────────────────────
load_dotenv()

# ─── 모델 초기화 ─────────────────────────
chat_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
rag_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

# ─── 1. 텍스트 추출 ────────────────────────
def extract_text_from_file(file_path: Path) -> str:
    suffix = file_path.suffix.lower()
    if suffix == ".pdf":
        loader = PyPDFLoader(str(file_path))
    elif suffix in (".docx", ".doc"):
        loader = Docx2txtLoader(str(file_path))
    else:
        raise ValueError("지원 파일 형식은 PDF 또는 DOCX 뿐입니다.")
    docs = loader.load()
    return "\n\n".join([d.page_content for d in docs])

# ─── 2. 벡터스토어 생성 ─────────────────────
def get_vectorstore(raw_text: str) -> FAISS:
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = splitter.split_documents([Document(page_content=raw_text)])
    return FAISS.from_documents(docs, embedding_model)

# ─── 3. 추천 직무 추출 ──────────────────────
recommend_template = PromptTemplate(
    input_variables=["resume_content"],
    template="""
당신은 커리어 분석 전문가입니다. 다음 이력서 내용을 기반으로, IT 직무에 국한하지 않고
지원자가 수행한 업무와 도구, 경험 등을 고려해 현실에 존재하는 직무 3가지를 추천하세요.

형식:
1. [직무명] - [간단한 설명]
2. ...
3. ...
"""
)

def recommend_jobs(vectorstore: FAISS, k: int = 4) -> str:
    docs = vectorstore.similarity_search("지원자의 업무 경험을 기반으로 추천 직무를 추론해줘", k=k)
    context = "\n\n".join([d.page_content for d in docs])
    prompt = recommend_template.format(resume_content=context)
    return rag_llm.predict(prompt)

# ─── 4. 관심 직무 기술 추출 ──────────────────
skill_template = PromptTemplate(
    input_variables=["position"],
    template="""
당신은 커리어 상담 전문가입니다.

직무명: "{position}"

이 직무는 일반적으로 어떤 업무를 수행하며, 이 업무를 성공적으로 수행하기 위해 반드시 필요한 핵심 기술 5가지를 아래 형식에 맞춰 작성하세요:

1. [기술명] - [간략한 설명 또는 사용 맥락]

조건:
- 현실에 존재하는 직무로 간주하고, 최대한 구체적인 기술명을 사용하세요.
- 예: Java, Spring Boot, REST API, MySQL, Docker, Jenkins 등
- 기술이 명확하지 않을 경우라도 일반적인 역량(예: 문제 해결, 테스트 자동화 등)을 포함하세요.
- 무조건 5개 항목을 출력하세요.

출력은 한국어로 하세요.
"""
)

def get_position_skills(position: str) -> str:
    prompt = skill_template.format(position=position)
    return chat_llm.predict(prompt)

# ─── 5. 유사도 계산 함수들 ──────────────────
def cosine_score(job: str, position: str) -> float:
    job_vec = embedding_model.embed_query(job)
    pos_vec = embedding_model.embed_query(position)
    return float(cosine_similarity([job_vec], [pos_vec])[0][0])

def cross_encoder_score(job: str, position: str) -> float:
    return float(cross_encoder.predict([(job, position)])[0])

def hybrid_score(job: str, position: str) -> tuple[float, float, float, float]:
    ce = cross_encoder_score(job, position)
    cos = cosine_score(job, position)

    # 자동 튜닝 α
    if len(position) >= 8 or "엔지니어" in position or "개발자" in position:
        alpha = 0.7
    else:
        alpha = 0.5

    cos_scaled = cos * 10
    final_score = alpha * ce + (1 - alpha) * cos_scaled
    return final_score, ce, cos, alpha

# ─── 6. 통합 분석 함수 ───────────────────────
def analyze_resume_and_position(
    raw_text: str,
    vectorstore: FAISS,
    position: str,
    k: int = 4,
    threshold: float = 6.0
) -> str:
    recommended = recommend_jobs(vectorstore, k=k).strip()
    rec_lines = [line.strip() for line in recommended.splitlines() if line.strip().startswith(('1', '2', '3'))]
    rec_jobs = [line.split(' ', 1)[1].split(' - ')[0].strip() for line in rec_lines]

    results = []
    best_score = -1.0
    for job in rec_jobs:
        score, ce, cos, alpha = hybrid_score(job, position)
        results.append((job, score, ce, cos, alpha))
        if score > best_score:
            best_score = score

    if best_score < threshold:
        return f"추천 직무와 관심 직무('{position}') 간 관련도가 낮아 분석이 어렵습니다.\n최고 유사도 점수: {best_score:.2f} (기준: {threshold})"

    skills = get_position_skills(position).strip()
    result = f"=== 관심 직무: {position} ===\n\n"
    result += "=== 추천 직무 및 유사도 점수 ===\n"
    for i, (job, score, ce, cos, alpha) in enumerate(results, 1):
        result += f"{i}. {job}\n"
        result += f"   - Hybrid Score: {score:.2f} | CrossEncoder: {ce:.2f} | Cosine: {cos:.2f} | α: {alpha:.2f}\n"

    result += "\n=== 관심 직무에 필요한 기술 ===\n" + skills
    return result

# ─── 7. CLI 진입점 ──────────────────────────
if __name__ == "__main__":
    path = Path(input("이력서 파일 경로(.pdf/.docx): ").strip())
    resume = extract_text_from_file(path)
    vectorstore = get_vectorstore(resume)

    position = input("관심 직무를 입력하세요 (예: 백엔드 개발자): ").strip()
    output = analyze_resume_and_position(
        raw_text=resume,
        vectorstore=vectorstore,
        position=position,
        k=4,
        threshold=6.8
    )

    print("\n=== 분석 결과 ===\n")
    print(output)


incorrect startxref pointer(1)
parsing for Object Streams



=== 분석 결과 ===

=== 관심 직무: 백엔드 엔지니어 ===

=== 추천 직무 및 유사도 점수 ===
1. 데이터 분석가
   - Hybrid Score: 7.59 | CrossEncoder: 7.18 | Cosine: 0.85 | α: 0.70
2. 디지털 마케팅 전문가
   - Hybrid Score: 6.74 | CrossEncoder: 6.03 | Cosine: 0.84 | α: 0.70
3. 프로젝트 매니저
   - Hybrid Score: 7.28 | CrossEncoder: 6.64 | Cosine: 0.88 | α: 0.70

=== 관심 직무에 필요한 기술 ===
1. Java - 백엔드 개발에 가장 널리 사용되는 프로그래밍 언어
2. Spring Framework - 응용 프로그램을 개발하고 실행하기 위한 프레임워크
3. RESTful API - 클라이언트와 서버 간의 통신을 위한 웹 서비스 아키텍처
4. SQL - 데이터베이스에서 데이터를 관리하고 조작하기 위한 언어
5. Git - 협업 및 버전 관리를 위한 분산 버전 관리 시스템.
